In [1]:
%matplotlib notebook

import json
import pyabc
import parse
import numpy as np
import scipy.stats as scs
import scipy.interpolate as sci
import matplotlib.pyplot as plt
import matplotlib.ticker as mpt

from ast import literal_eval

In [2]:
base_parameters = {
    'k_a': 0.002,
    'k_d': 0.1,
    'mu': 3.0,
    'kappa': 1.0,
    'gamma': 0.04,
    'diffusion': 0.6,
    'time_step': 0.1,
    'cell_radius': 6.0,
    'nucleus_radius': 2.5,
}

In [3]:
with open('time.json', 'r') as f:
    raw_data = {literal_eval(k): v for k, v in json.load(f).items()}

In [4]:
# Refactor data
data = {}
for k, v in raw_data.items():
    solver, params = k
    data.setdefault(solver, {})[params] = v

In [22]:
for solver in ['WMM', 'CBM', 'smoldyn']:
    time = data[solver]
    min_x = min(time.keys(), key=lambda t: t[1])[1]
    max_x = max(time.keys(), key=lambda t: t[1])[1]

    min_D = min(time.keys(), key=lambda t: t[0])[0]
    max_D = max(time.keys(), key=lambda t: t[0])[0]

    raw_grid = np.array([[chi, D, z[1]]
                         for ((D, chi, k_d), z) in time.items()
                        if k_d == 0])

    XY = raw_grid[:, :2]
    Z = raw_grid[:, 2]


    inference_files = sorted([
        "inference/inference_0.8000000000000007_0.8000000000000003_0.0_.db",
        "inference/inference_-0.7999999999999998_0.8000000000000003_0.0_.db",
        "inference/inference_-4.8_3.2_0.0_.db",
    ], key=lambda s: -parse.parse("inference/inference_{diffusion:f}_{chi:f}_{k_d:f}_.db", s).named['diffusion'])

    print(f"{solver} (s): ".ljust(13), end='')
    for inference_file in inference_files:
        db_path = "sqlite:///{}".format(inference_file)
        history = pyabc.History(db_path)

        particles = np.array([[particle.parameter['chi'], particle.parameter['diffusion']]
            for t in range(10)
            for particle in history.get_population(t=t).get_list()])

        xi, yi = np.mgrid[min_x:max_x:100j, min_D:max_D:100j]
        grid_z2 = sci.griddata(XY, Z, (particles[:, 0], particles[:, 1]) , method='cubic')

        print(f"{sum(grid_z2)/60./60.:9.2f}", end=' ')
    print()

WMM (s):          1.70      1.47      3.49 
CBM (s):          2.21      1.79      1.14 
smoldyn (s):   4635.48   3405.71    680.40 


In [19]:
# get particle coordinates
# check that time is realistic
print(0.6*2**0.8, 2**0.8)
print(0.6*2**-0.8, 2**0.8)
print(0.6*2**-4.8, 2**3.2)

1.044660675955349 1.7411011265922482
0.34460950649911043 1.7411011265922482
0.02153809415619441 9.18958683997628
